In [1]:
import sys
sys.path.append("ALL_IN_ONE/Pythia8")
sys.path.append("ALL_IN_ONE/LSD")
sys.path.append("ALL_IN_ONE/FIT_FUNC")

In [2]:
import os
from tqdm import tqdm
import ALL_IN_ONE.Pythia8.loop as loop
import ALL_IN_ONE.Pythia8.one_key_run as okr 
import ALL_IN_ONE.Pythia8.run_save as rs
import ALL_IN_ONE.Pythia8.analyse_data as ad
import ALL_IN_ONE.Pythia8.detection as detection
import ALL_IN_ONE.Pythia8.Mulity_process as mp
from datetime import datetime
import ALL_IN_ONE.Pythia8.combine as combine
# from detection import detect_percise_and_combine, detect_r_and_combine
# from analyse_data import see_ctau_2, print_max_min_log10_sin_theta_2

### This Func is made for FUTURE USE aimmed to speed up our simulations(Whose speed is Not Satisfying right now(2024.11.17)). 

### But it is not usable now

In [3]:
# from joblib import Parallel, delayed

# def worker(seed):
#     LLP_data_path = loop_mass_ctau_given_by_csv_multi(csv_file, br, seed, out_put_path, main41_path)
#     print('The Generation of LLPs is Completed')
#     completed_data_dir = detect_folder_files_no_calcu(LLP_data_path)[1]
#     print('The LLPs are Judged whether they are Detected or not')
#     final_files = combine_files_precise(completed_data_dir)
#     print('The Final Step is Over, See the .csv files for LLPs Completed Data')
#     return LLP_data_path, completed_data_dir, final_files

# def one_key_run_mass_ctau_given_by_csv_multi(csv_file, br, seed_array, out_put_path, main41_path):
#     results = Parallel(n_jobs=5)(delayed(worker)(seed) for seed in seed_array)
#     return results


## Attention:  

## The Units are CM and GEV

### The 'one_key_run_XXX' functions are all you need to use.



### Setting Parameters

In [4]:
c_mm = (3*10**5) # tau0's unit is mm/c, to convert the ultimate decay postion,
                            # we need to do sth to the ctau_typed_in 

br_lowerlim = -20 #ctau and br are in logspace
br_upperlim = -20
br_amount = 1
ctau_lowerlim = 1
ctau_upperlim = 30
ctau_amount = 50
mass = 1
seed_amount = 5
seed_array = (357993, 1234) #Pythia8 simulation seed

mass_lower_lim = 0.1 # Mass in in linspace
mass_upper_lim = 4
mass_amount = 20
br = 1

#### Same as before:  

##### The function 
```python
    one_key_run_mass_ctau_simple_ver() #Function to run an pythia8 program
```

takes 

```
br's lower / upper lim (10^x), br's amount, 

ctau's lower / upper lim (10^x), ctau's amount, 

mass, 

seed's amount, 

the Output folder's path, 

main41.cc's dir path
```
as input.

The Output is LOTS OF ```.csv``` Files with LLPs' Properties 

```
    Momentum in GeV, 
    
    lifetime(ctau or tau) in mm/C, 

    decay_postion in mm, mass in GeV 

    and so on
```

The path's format is listed below, you can change them directly to your real path to run the function

In [5]:
main131_path = '/media/ubuntu/6156e08b-fdb1-4cde-964e-431f74a6078e/Program/pythia8312/examples/'
# For Pythia8.31, main41 is now main131 but the path remains the same
out_path = '/media/ubuntu/6156e08b-fdb1-4cde-964e-431f74a6078e/Files/LLP_DATA/Original_DATA/'
today = str(datetime.now().date())+"-2HDM"
print(today)

2025-04-23-2HDM


#### New Function info:

The function
```python
    def one_key_run_mass_ctau_br_given_by_csv_main131(csv_file, br, seed_amount, out_path, main41_path)
```

takes a ```.csv file``` as input instead of the range of ```mass```, ```tau```, ```Br_XX```and etc. 

The function will read each line of ```.csv file``` to get `LLP's DATA Range`, ALONG WITH ```ALL THE Branching Ratio of OUR LLP``` and put this set of data into pythia8. 

Pythia8 will do the simulation, then output .csv file to the ``` out_path``` folder.

In [ ]:
csv_file = '/media/ubuntu/6156e08b-fdb1-4cde-964e-431f74a6078e/Program/Light_scalar_decay/Combined_Code/2HDM_test_1.csv'
mp.split_csv_into_different_parts(csv_file, 3)

In [9]:
test_for_mass_1 = okr.one_key_run_2HDM_cross_section_main131_lower_eff_all_detectors(csv_file, 1, 1, out_path, main131_path, today, 0)

Running Simulation...


  0%|          | 0/2961 [00:00<?, ?it/s]

100%|██████████| 2961/2961 [68:06:17<00:00, 82.80s/it]   


The Generation of LLPs is Completed


100%|██████████| 3/3 [00:00<00:00, 6557.02it/s]


UnboundLocalError: local variable 'completed_data_folder' referenced before assignment

In [ ]:
today = str(datetime.now().date())+"-2HDM"

print(today)
test_for_mass_1 = okr.one_key_run_2HDM_cross_section_main131_lower_eff_all_detectors(csv_file, 1, 1, out_path, main131_path, today, 0)

In [ ]:
from concurrent.futures import ProcessPoolExecutor
import os

# 假设 okr.one_key_run_by_csv_cross_section_main131 是你要并行运行的函数
from concurrent.futures import ProcessPoolExecutor

def run_task(args):
    _csv_file, _param1, _param2, _out_path, _main131_path, _date, _sleep_time = args
    return okr.one_key_run_2HDM_cross_section_main131_lower_eff_all_detectors(_csv_file, _param1, _param2, _out_path, _main131_path, _date, _sleep_time)

if __name__ == "__main__":
    # 自定义 CPU 核心数量
    num_cores = 3  # 你可以将此值改为你想使用的核心数量

    # 定义任务参数
    # out_path = "output_path"  # 替换为实际的输出路径
    # main131_path = "main131_path"  # 替换为实际的路径

    tasks = [
        (f"{i+1}.csv", 1, 2, out_path, main131_path, today, 10)  # 生成任务参数
        for i in range(num_cores)  # 根据自定义核心数生成任务
    ]

    # 打印生成的任务
    for task in tasks:
        print(task)

    # 使用多核并行
    with ProcessPoolExecutor(max_workers=num_cores) as executor:
        results = list(executor.map(run_task, tasks))

    # 打印结果
    print("All tasks completed:", results)


run_task((csv_file, 1, 1, out_path, main131_path, today, 0))

('1.csv', 1, 3, '/media/ubuntu/6156e08b-fdb1-4cde-964e-431f74a6078e/Files/LLP_DATA/Original_DATA/', '/media/ubuntu/6156e08b-fdb1-4cde-964e-431f74a6078e/Program/pythia8312/examples/', '2025-04-19', 10)
('2.csv', 1, 3, '/media/ubuntu/6156e08b-fdb1-4cde-964e-431f74a6078e/Files/LLP_DATA/Original_DATA/', '/media/ubuntu/6156e08b-fdb1-4cde-964e-431f74a6078e/Program/pythia8312/examples/', '2025-04-19', 10)
('3.csv', 1, 3, '/media/ubuntu/6156e08b-fdb1-4cde-964e-431f74a6078e/Files/LLP_DATA/Original_DATA/', '/media/ubuntu/6156e08b-fdb1-4cde-964e-431f74a6078e/Program/pythia8312/examples/', '2025-04-19', 10)
('4.csv', 1, 3, '/media/ubuntu/6156e08b-fdb1-4cde-964e-431f74a6078e/Files/LLP_DATA/Original_DATA/', '/media/ubuntu/6156e08b-fdb1-4cde-964e-431f74a6078e/Program/pythia8312/examples/', '2025-04-19', 10)
Running Simulation...Running Simulation...Running Simulation...Running Simulation...





100%|██████████| 2850/2850 [32:51:20<00:00, 41.50s/it]   


The Generation of LLPs is Completed


100%|██████████| 2850/2850 [32:51:30<00:00, 41.51s/it]


The Generation of LLPs is Completed


...

The Final Step is Over, See the .csv files for LLPs Completed DataThe Final Step is Over, See the .csv files for LLPs Completed Data



100%|██████████| 11601/11601 [58:13<00:00,  3.32it/s]


The Final Step is Over, See the .csv files for LLPs Completed Data
The Final Step is Over, See the .csv files for LLPs Completed Data
All tasks completed: [('/media/ubuntu/6156e08b-fdb1-4cde-964e-431f74a6078e/Files/LLP_DATA/Original_DATA/2025-04-19/LLP_data', '/media/ubuntu/6156e08b-fdb1-4cde-964e-431f74a6078e/Files/LLP_DATA/Original_DATA/2025-04-19/Completed_llp_data_precise_cross_section', ('/media/ubuntu/6156e08b-fdb1-4cde-964e-431f74a6078e/Files/LLP_DATA/Original_DATA/2025-04-19/2025-04-20_all_combined_precise_file.csv', '/media/ubuntu/6156e08b-fdb1-4cde-964e-431f74a6078e/Files/LLP_DATA/Original_DATA/2025-04-19/2025-04-20_detected_combined_precise_file.csv')), ('/media/ubuntu/6156e08b-fdb1-4cde-964e-431f74a6078e/Files/LLP_DATA/Original_DATA/2025-04-19/LLP_data', '/media/ubuntu/6156e08b-fdb1-4cde-964e-431f74a6078e/Files/LLP_DATA/Original_DATA/2025-04-19/Completed_llp_data_precise_cross_section', ('/media/ubuntu/6156e08b-fdb1-4cde-964e-431f74a6078e/Files/LLP_DATA/Original_DATA/2025-0

 11%|█         | 408/3800 [7:39:40<63:41:39, 67.60s/it]


KeyboardInterrupt: 

#### To read the data

Now that you have done the simulation via pythia8, the output .csv files should be in your `output` folder. 

Below the folder, the file is sorted and filed by `date of the simulation`, so there will be a new folder like `2025-03-07`. In that `date folder`, there will be two folders, named`LLP_data`(store the orginal data), and `Completed_llp_data_precise` (store the detective data). 

AND THERE IS A `.csv` FILE, which is named like `2025-03-07_detected_combined_precise_file.csv`, it contains ONLY the LLPs can be DETECTED by CODEX-b.

In [ ]:
print(ad.print_max_min_log10_sin_theta_2('/Users/shiyuzhe/Documents/University/LLP/Second_Term/pythia8/BtoKa/Code/position_read_analyse_1.3/LLP_data_for_test/2025-2-16/2025-2-16_detected_combined_precise_file.csv'))

(-10.200000256677525, -12.400000474218762)


### The Detect Function

Functions::::
```python
def detect_r_and_combine(LLP_data_path) 

def detect_precise_and_combine(LLP_data_path)
```

Are functions to combine '''.csv files''' and judge whether LLP can be detected by CODEX-b or not

They will creat a ```.csv file``` at the dir of your LLP, which contains ALL DETECTED LLPS.

The function::::
```python 
def detect_r_and_combine(LLP_data_path)
```
 is merely a test function. You can play with it to get more data, but DO NOT use it for real analyzation

In [10]:
LLP_date_path = '/media/ubuntu/6156e08b-fdb1-4cde-964e-431f74a6078e/Files/LLP_DATA/Original_DATA/2025-04-23-2HDM/LLP_data/B_2HDM/'
# detected_r = '/Users/shiyuzhe/Documents/University/LLP/Second_Term/pythia8/BtoKa/Code/position_read_analyse_1.3/LLP_data_for_test/2024-11-18_19/'
okr.calcu_cross_section_and_combine_files_CODEX_MATHUSLA(LLP_date_path)

  0%|          | 4/2961 [00:01<19:13,  2.56it/s]

Error with file: /media/ubuntu/6156e08b-fdb1-4cde-964e-431f74a6078e/Files/LLP_DATA/Original_DATA/2025-04-23-2HDM/LLP_data/B_2HDM/mass_7.94e+00_ctau_4.00e+06_br_1.00e+00_seed_979224.csv
Error message: No columns to parse from file


  0%|          | 6/2961 [00:01<13:41,  3.60it/s]

Error with file: /media/ubuntu/6156e08b-fdb1-4cde-964e-431f74a6078e/Files/LLP_DATA/Original_DATA/2025-04-23-2HDM/LLP_data/B_2HDM/mass_6.31e+00_ctau_4.00e+07_br_1.00e+00_seed_979224.csv
Error message: No columns to parse from file


  1%|          | 17/2961 [00:05<17:45,  2.76it/s]

...

100%|██████████| 2961/2961 [18:18<00:00,  2.70it/s]


The LLPs are Judged whether they are Detected or not, and calculated the cross section


100%|██████████| 2398/2398 [06:01<00:00,  6.63it/s]

The Final Step is Over, See the .csv files for LLPs Completed Data


('/media/ubuntu/6156e08b-fdb1-4cde-964e-431f74a6078e/Files/LLP_DATA/Original_DATA/2025-04-23-2HDM/LLP_data/Completed_llp_data_precise_cross_section',
 ('/media/ubuntu/6156e08b-fdb1-4cde-964e-431f74a6078e/Files/LLP_DATA/Original_DATA/2025-04-23-2HDM/LLP_data/2025-04-29_all_combined_precise_file.csv',
  '/media/ubuntu/6156e08b-fdb1-4cde-964e-431f74a6078e/Files/LLP_DATA/Original_DATA/2025-04-23-2HDM/LLP_data/2025-04-29_detected_combined_precise_file.csv'))

In [ ]:
combinefile = '/media/ubuntu/6156e08b-fdb1-4cde-964e-431f74a6078e/Files/LLP_DATA/Original_DATA/'
combine.merge_csv_files(combinefile, combinefile + '/combined.csv')

All CSV files have been merged into /media/ubuntu/6156e08b-fdb1-4cde-964e-431f74a6078e/Files/LLP_DATA/Original_DATA/ALL_GOOD_DATA_2/combined.csv
